In [ ]:
#Load libs

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from pathlib import Path
from tysserand import tysserand as ty
from PIL import Image
Image.MAX_IMAGE_PIXELS = 1000000000 
from PIL import Image, ImageOps
import fcsparser

from os import listdir
from os.path import isfile, join
#set up working dir

import sys
sys.path.extend([
    '../tysserand/tysserand',
    '../mosna',
])

import seaborn as sns
from time import time
import copy
from skimage import color
import matplotlib as mpl
import napari

from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_validate, GridSearchCV, RandomizedSearchCV
from scipy.stats import loguniform

import umap
# if not installed run: conda install -c conda-forge umap-learn
import hdbscan
from sklearn.cluster import OPTICS, cluster_optics_dbscan

from skimage import io

from scipy.stats import ttest_ind    # Welch's t-test
from scipy.stats import mannwhitneyu # Mann-Whitney rank test
from scipy.stats import ks_2samp     # Kolmogorov-Smirnov statistic

sys.path.append("/home/mouneem/mosna/")


from tysserand import tysserand as ty
from mosna import mosna






data_dir = Path("/home/mouneem/tysserand/CORDELIER_PIERRE/")
path = str("/home/mouneem/tysserand/CORDELIER_PIERRE/")
path_nets = str("/home/mouneem/tysserand/CORDELIER_PIERRE/nets/")
#path = str("/home/mouneem/tysserand/data/")

# select all CSVs
csv_path = path+"csv_levels/"
all_csvs = [f for f in listdir(csv_path) if isfile(join(csv_path, f))]

#  img
fileimg = "C2v1/NVA_21-003.IMMCORE.C2v1_15T008187-08-ImvessC2-4321.jpg"


filename_pattern = ".".join(fileimg.split('/')[-1].split('.')[0:-1])
## filtre by pattern
csv_levels = [csv_file for csv_file in all_csvs if filename_pattern in csv_file ]
print(csv_levels)


save_dir = Path("/home/mouneem/tysserand/CORDELIER_PIERRE/mosna_output")
mosna_output = str("/home/mouneem/tysserand/CORDELIER_PIERRE/mosna_output/")

sns.set_context('talk')


In [ ]:

for csv_file in csv_levels[::1]:

    print(csv_file)
    nodes = pd.read_csv(csv_path + csv_file , header = None)
    nodes.columns = ['id','x','y','class']

    ratio = 1
    nodes = nodes.sample(n = round(len(nodes) / ratio) )
    coords = nodes.loc[:,['x','y']].values

    #load image
    img = plt.imread(data_dir / fileimg)

    #make colors
    nodes['color'] = '#7f8c8d'
    nodes.loc[nodes['class'] == 'C1' , 'color' ] = '#e74c3c' #RED
    nodes.loc[nodes['class'] == 'C1' , 'color' ] = '#e67e22' #ORANGE
    nodes.loc[nodes['class'] == 'C1' , 'color' ] = '#f1c40f' #YELLOW
    nodes.loc[nodes['class'] == 'C1' , 'color' ] = '#8e44ad' #PURPLE
    nodes.loc[nodes['class'] == 'C1' , 'color' ] = '#2980b9' #LIGHT BLUE 
    nodes.loc[nodes['class'] == 'C1' , 'color' ] = '#16a085' #GREEN SEA
    nodes.loc[nodes['class'] == 'C1' , 'color' ] = '#2c3e50' #BLUE MIDNIGHT
    nodes.loc[nodes['class'] == 'C1' , 'color' ] = '#27ae60' #GREEN
    nodes.loc[nodes['class'] == 'C1' , 'color' ] = '#7f8c8d' #GRAY


    #CROP
    w = img.shape[0]
    h = img.shape[1]
    # Cropping an image
    x1 = int(w * .0)
    x2 = int(w * 1)
    y1 = int(h * 0)
    y2 = int(h * 1)
    cropped_image = img[x1:x2, y1:y2]
    mask = (nodes['y'] > x1*10) & (nodes['y'] < x2*10  ) &  (nodes['x'] > y1 *10 ) &  (nodes['x'] < y2 *10 )
    cropped_coords = nodes.loc[mask,['x','y']].values
    print(type(cropped_coords))

    cropped_colors = nodes['color']



    from collections import Counter

    ##########
    cropped_coords[:,0]  = (cropped_coords[:,0] * 0.1).round()
    cropped_coords[:,1]  = (cropped_coords[:,1] * 0.1).round()

    fileimgPng = csv_file+".png"

    fig, ax = ty.showim(cropped_image, figsize=(100, 100))
    cropped_pairs = ty.build_delaunay(cropped_coords)
    ty.plot_network(cropped_coords, cropped_pairs, col_nodes=cropped_colors , size_nodes=1, col_edges='w', ax=ax, linewidth = 1, alpha_edges = .5)


    fig.savefig(path_nets+"full_"+fileimgPng)
    print(path_nets+"full_"+fileimgPng)



    nodesAll = nodes


    nodes = nodesAll
    nodes.loc[nodes['class'] == 'C1' , 'class' ] = 'Cancer' # 'PanCK+CD3-CD8-CD20-'
    nodes.loc[nodes['class'] == 'C2' , 'class' ] = 'CD8 T-Cell' # 'PanCK-CD3+CD8+' 
    nodes.loc[nodes['class'] == 'C3' , 'class' ] = 'CD4 T-Cell' # 'PanCK-CD3+CD8-' 
    nodes.loc[nodes['class'] == 'C4' , 'class' ] =  'B Cell' #'PanCK-CD3-CD20+' 
    nodes.loc[nodes['class'] == 'C5' , 'class' ] = 'CD3+CD20+' # 'PanCK-CD3+CD20+' 

    attributes = nodes['class']

    edges = pd.DataFrame(data=cropped_pairs, columns=['source', 'target'])
    nodes = pd.DataFrame(data=cropped_coords, columns=['x', 'y'])
    nodes['Group'] = attributes.values
    nodes = nodes.join(pd.get_dummies(nodes['Group']))
    att = nodes['Group']
    attributes = nodes['Group'].unique()
    nodes.head()


    # Network statistics
    mixmat = mosna.mixing_matrix(nodes, edges, attributes)
    assort = mosna.attribute_ac(mixmat)

    # ------ Randomization ------
    mixmat_rand, assort_rand = mosna.randomized_mixmat(nodes, edges, attributes, n_shuffle=20, parallel=False)
    mixmat_mean, mixmat_std, mixmat_zscore = mosna.zscore(mixmat, mixmat_rand, return_stats=True)
    assort_mean, assort_std, assort_zscore = mosna.zscore(assort, assort_rand, return_stats=True)

    mixmat = mosna.mixmat_to_df(mixmat, attributes)
    mixmat_zscore = mosna.mixmat_to_df(mixmat_zscore, attributes)




    title = "Assortativity by cell types:"
    print(title)
    fig, ax = plt.subplots(figsize=(9, 6))
    sns.heatmap(mixmat, center=0, cmap="vlag", annot=True, linewidths=.5, ax=ax)
    plt.xticks(rotation=30, ha='right');
    # plt.xticks(rotation=30, ha='right', fontsize=20);
    # plt.yticks(fontsize=20);
    plt.savefig(mosna_output + csv_file+"assortativity.png", bbox_inches='tight', facecolor='white')



In [ ]:



nodes = nodesAll
nodes.loc[nodes['class'] == 'C1' , 'class' ] = 'PanCK+CD3-CD8-CD20-'
nodes.loc[nodes['class'] == 'C2' , 'class' ] = 'PanCK-CD3+CD8+' 
nodes.loc[nodes['class'] == 'C3' , 'class' ] = 'PanCK-CD3+CD8-' 
nodes.loc[nodes['class'] == 'C4' , 'class' ] = 'PanCK-CD3-CD20+' 
nodes.loc[nodes['class'] == 'C5' , 'class' ] = 'PanCK-CD3+CD20+' 

attributes = nodes['class']

edges = pd.DataFrame(data=cropped_pairs, columns=['source', 'target'])
nodes = pd.DataFrame(data=cropped_coords, columns=['x', 'y'])
nodes['Group'] = attributes.values
nodes = nodes.join(pd.get_dummies(nodes['Group']))
att = nodes['Group']
attributes = nodes['Group'].unique()
nodes.head()



# Network statistics
mixmat = mosna.mixing_matrix(nodes, edges, attributes)
assort = mosna.attribute_ac(mixmat)

# ------ Randomization ------
mixmat_rand, assort_rand = mosna.randomized_mixmat(nodes, edges, attributes, n_shuffle=200, parallel=False)
mixmat_mean, mixmat_std, mixmat_zscore = mosna.zscore(mixmat, mixmat_rand, return_stats=True)
assort_mean, assort_std, assort_zscore = mosna.zscore(assort, assort_rand, return_stats=True)

mixmat = mosna.mixmat_to_df(mixmat, attributes)
mixmat_zscore = mosna.mixmat_to_df(mixmat_zscore, attributes)




In [ ]:
save_dir = Path("/home/mouneem/tysserand/CORDELIER_PIERRE/mosna_output")

sns.set_context('talk')

title = "Assortativity by cell types:"
print(title)
fig, ax = plt.subplots(figsize=(9, 6))
sns.heatmap(mixmat, center=0, cmap="vlag", annot=True, linewidths=.5, ax=ax)
plt.xticks(rotation=30, ha='right');
# plt.xticks(rotation=30, ha='right', fontsize=20);
# plt.yticks(fontsize=20);
plt.savefig(save_dir / "assortativity", bbox_inches='tight', facecolor='white')

print("Z-scored assortativity by cell types: {}".format(assort_zscore))
fig, ax = plt.subplots(figsize=(9, 6))
sns.heatmap(mixmat_zscore, center=0, cmap="vlag", annot=True, linewidths=.5, ax=ax)
plt.xticks(rotation=30, ha='right');
plt.savefig(save_dir / "assortativity_z-scored", bbox_inches='tight', facecolor='white')